In [ ]:
import pandas as pd
import json
import os

INPUT_FILE = 'terrorism_dataset.csv' 
OUTPUT_FILE = 'global_attacks.json'

def clean_attack_type(text):
    if pd.isna(text):
        return "unknown"
    
    text = text.lower()
    
    if "bombing" in text or "explosion" in text:
        return "explosion"
    
    return text.replace(' ', '_').replace('/', '_')

def generate_json_from_gtd():
    print("lOADING dATASET...")
    
    try:
        df = pd.read_csv(INPUT_FILE, encoding='ISO-8859-1', low_memory=False)
    except FileNotFoundError:
        print(f"Errore: File {INPUT_FILE} non trovato.")
        return

    df_reduced = df[['iyear', 'country_txt', 'attacktype1_txt']].copy()
    df_reduced.columns = ['year', 'country', 'attack_type']

    print("Data Elaboration")

    grouped = df_reduced.groupby(['year', 'country', 'attack_type']).size().reset_index(name='count')
    grouped.sort_values(['year', 'country', 'count'], ascending=[True, True, False], inplace=True)
    most_frequent = grouped.drop_duplicates(subset=['year', 'country'])

    final_dict = {}

    for _, row in most_frequent.iterrows():
        year = str(row['year'])
        country = row['country']
        attack = clean_attack_type(row['attack_type'])

        if year not in final_dict:
            final_dict[year] = {}
        
        final_dict[year][country] = attack

    print(f"Saving file {OUTPUT_FILE}...")
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_dict, f, indent=1, ensure_ascii=False)

    print("Done!")
if __name__ == "__main__":
    generate_json_from_gtd()

Caricamento del dataset in corso...
Elaborazione dei dati...
Salvataggio del file global_attacks.json...
Fatto!
